# Validation and overfitting

퍼블릭 리더보드에서 프라이빗 리더보드로 갔을 때 순위가 팍 떨어지는 경우가 있다.

1. validation을 안하고 퍼블릭에서 제일 점수 높은거로 낸 경우
2. 퍼블릭/프라이빗을 이상하게 나눈 경우


우리가 신경쓸 것은 1번. 이것을 competition에서의 overfitting이라고 한다.

- 일반적인 의미에서의 overfitting

training error << test error

- competition에서의 overfitting

기대한 error >> test error

# Validation strategies